In [1]:
import sys,os
if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))
from dashgrid.dgrid_components import PlotlyCandles as plc
from dashgrid import db_info as dbi

import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output,State
from dash.exceptions import PreventUpdate
import dash_table
import pandas as pd
import numpy as np
import json
import logging
import datetime
import functools
import random
import inspect
from pandasql import sqldf
import datetime,base64,io,pytz


In [2]:
DEFAULT_LOG_PATH = './logfile.log'
DEFAULT_LOG_LEVEL = 'INFO'

def init_root_logger(logfile=DEFAULT_LOG_PATH,logging_level=DEFAULT_LOG_LEVEL):
    level = logging_level
    if level is None:
        level = logging.DEBUG
    # get root level logger
    logger = logging.getLogger()
    if len(logger.handlers)>0:
        return logger
    logger.setLevel(logging.getLevelName(level))

    fh = logging.FileHandler(logfile)
    fh.setLevel(logging.DEBUG)
    # create console handler with a higher log level
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    # create formatter and add it to the handlers
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    ch.setFormatter(formatter)
    # add the handlers to the logger
    logger.addHandler(fh)
    logger.addHandler(ch)   
    return logger


In [3]:
logger = init_root_logger(logging_level='DEBUG')


In [4]:
def stop_callback(errmess,logger=None):
    m = "****************************** " + errmess + " ***************************************"     
    if logger is not None:
        logger.debug(m)
    raise PreventUpdate()


# Dlink: Link Dash components 

In [5]:
def slice_df(df_in,input_list):
    """!
        ### slice_df slices a DataFrame according to a list of values.
        1. The arg ```input_list``` contains values that coincide with columns of the DataFrame
        2. The method returns a new filtered DataFrame where:
         * the method filters each column using the respective value in ```input_list```
         * if there is a ```None``` in any item in the input list, the method ignores filtering on that column
        :param df_in: DataFrame to be sliced
        :param input_list: list of values to use as criteria for each column slice. 
                            A value of None will cause the method to not slice that column
        :return: new DataFrame that has been sliced
    """
    dfc = df_in.drop_duplicates().copy()
    cols = dfc.columns.values
    current_input = 0
    for i in range(len(input_list)):
        il = input_list[i]
        col = cols[i]
        if il is not None:
            try:                
                dfc = dfc[dfc[col]==il]
            except Exception as e:
                raise ValueError(e)
        current_input += 1
    return dfc


In [6]:
def choices_from_df(df_in,input_list,col_to_select):
    '''
    Create a list of dicts that conform to an options list for a Dash componet like RadioItems or Dropdown,where:
    1. Each dict in the list has a 'label' key, and a 'value' key
    2. ```df_in``` is sliced
    '''
    dfc = slice_df(df_in,input_list)
    if dfc is None or len(dfc)<1:
        return None,None
    unique_values = dfc[col_to_select].unique()
    choices = unique_values[0],[{'label':uv,'value':uv} for uv in unique_values]
    return choices



In [7]:
GRID_STYLE = {'display': 'grid',
              'border': '1px solid #000',
              'grid-gap': '8px 8px',
              'background-color':'#fffff9',
            'grid-template-columns': '1fr 1fr'}

def create_grid(
        component_array,num_columns=2,
        column_width_percents=None,
        additional_grid_properties_dict=None,
        wrap_in_loading_state=False,
        row_layout=None):
    gs = GRID_STYLE.copy()
    if row_layout is None:
        percents = [str(round(100/num_columns-.001,1))+'%' for _ in range(num_columns)] if column_width_percents is None else [str(c)+'%' for c in column_width_percents]
        perc_string = " ".join(percents)
        gs['grid-template-columns'] = perc_string 
    else:
        gs['grid-template-columns'] = row_layout
    if additional_grid_properties_dict is not None:
        for k in additional_grid_properties_dict.keys():
            gs[k] = additional_grid_properties_dict[k]           
    
    div_children = []
    for c in component_array:
        if type(c)==str:
            div_children.append(GridItem(c).html)
        elif hasattr(c,'html'):
            div_children.append(c.html)
        else:
            div_children.append(c)
    if wrap_in_loading_state:
        g = dcc.Loading(html.Div(div_children,style=gs),type='cube')
    else:
        g = html.Div(div_children,style=gs)
    return g

class GridItem():
    def __init__(self,child,html_id=None):
        self.child = child
        self.html_id = html_id
    @property
    def html(self):
        if self.html_id is not None:
            return html.Div(children=self.child,className='grid-item',id=self.html_id)
        else:
            return html.Div(children=self.child,className='grid-item')


In [8]:
#********** define useful css styles ***********************
borderline = 'none' #'solid'
button_style={
    'line-height': '40px',
    'borderWidth': '1px',
    'borderStyle': borderline,
    'borderRadius': '1px',
    'textAlign': 'center',
    'background-color':'#fffff0',
    'vertical-align':'middle',
}
button_style_no_border={
    'line-height': '40px',
    'textAlign': 'center',
    'background-color':'#fffff0',
    'vertical-align':'middle',
}

blue_button_style={
    'line-height': '40px',
    'textAlign': 'center',
    'background-color':'#A9D0F5',#ed4e4e
    'vertical-align':'middle',
}

border_style={
    'line-height': '40px',
    'border':borderline + ' #000',
    'textAlign': 'center',
    'vertical-align':'middle',
}

table_like = {
    'display':'table',
    'width': '100%'
}

# define h4_like because h4 does not play well with dash_table
h4_like = { 
    'display': 'table-cell',
    'textAlign' : 'center',
    'vertical-align' : 'middle',
    'font-size' : '16px',
    'font-weight': 'bold',
    'width': '100%',
    'color':'#22aaff'
}


cte_title_style = { 
    'textAlign' : 'center',
    'vertical-align' : 'middle',
    'font-weight': 'bold',
    'color':'#22aaff'
}

DEFAULT_TIMEZONE = 'US/Eastern'


In [9]:
# ************************* define useful factory methods *****************

def parse_contents(contents):
    '''
    app.layout contains a dash_core_component object (dcc.Store(id='df_memory')), 
      that holds the last DataFrame that has been displayed. 
      This method turns the contents of that dash_core_component.Store object into
      a DataFrame.
      
    :param contents: the contents of dash_core_component.Store with id = 'df_memory'
    :returns pandas DataFrame of those contents
    '''
    c = contents.split(",")[1]
    c_decoded = base64.b64decode(c)
    c_sio = io.StringIO(c_decoded.decode('utf-8'))
    df = pd.read_csv(c_sio)
    # create a date column if there is not one, and there is a timestamp column instead
    cols = df.columns.values
    cols_lower = [c.lower() for c in cols] 
    if 'date' not in cols_lower and 'timestamp' in cols_lower:
        date_col_index = cols_lower.index('timestamp')
        # make date column
        def _extract_dt(t):
            y = int(t[0:4])
            mon = int(t[5:7])
            day = int(t[8:10])
            hour = int(t[11:13])
            minute = int(t[14:16])
            return datetime.datetime(y,mon,day,hour,minute,tzinfo=pytz.timezone(DEFAULT_TIMEZONE))
        # create date
        df['date'] = df.iloc[:,date_col_index].apply(_extract_dt)
    return df

def make_df(dict_df):
    if type(dict_df)==list:
        if type(dict_df[0])==list:
            dict_df = dict_df[0]
        return pd.DataFrame(dict_df,columns=dict_df[0].keys())
    else:
        return pd.DataFrame(dict_df,columns=dict_df.keys())

class BadColumnsException(Exception):
    def __init__(self,*args,**kwargs):
        Exception.__init__(self,*args,**kwargs)



def create_dt_div(dtable_id,df_in=None,
                  columns_to_display=None,
                  editable_columns_in=None,
                  title='Dash Table',logger=None,
                  title_style=None):
    '''
    Create an instance of dash_table.DataTable, wrapped in an dash_html_components.Div
    
    :param dtable_id: The id for your DataTable
    :param df_in:     The pandas DataFrame that is the source of your DataTable (Default = None)
                        If None, then the DashTable will be created without any data, and await for its
                        data from a dash_html_components or dash_core_components instance.
    :param columns_to_display:    A list of column names which are in df_in.  (Default = None)
                                    If None, then the DashTable will display all columns in the DataFrame that
                                    it receives via df_in or via a callback.  However, the column
                                    order that is displayed can only be guaranteed using this parameter.
    :param editable_columns_in:    A list of column names that contain "modifiable" cells. ( Default = None)
    :param title:    The title of the DataFrame.  (Default = Dash Table)
    :param logger:
    :param title_style: The css style of the title. Default is dgrid_components.h4_like.
    '''
    # create logger 
    lg = init_root_logger() if logger is None else logger
    
    lg.debug(f'{dtable_id} entering create_dt_div')
    
    # create list that 
    editable_columns = [] if editable_columns_in is None else editable_columns_in
    datatable_id = dtable_id
    dt = dash_table.DataTable(
        page_current= 0,
        page_size= 100,
        filter_action='none', # 'fe',
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ],
        style_cell_conditional=[
            {
                'if': {'column_id': c},
                'textAlign': 'left',
            } for c in ['symbol', 'underlying']
        ],

        style_as_list_view=False,
        style_table={
            'maxHeight':'450px','overflowX': 'scroll','overflowY':'scroll'
#             'height':'15','overflowX': 'scroll','overflowY':'scroll'
        } ,
        editable=True,
        css=[{"selector": "table", "rule": "width: 100%;"}],
        id=datatable_id
    )
    if df_in is None:
        df = pd.DataFrame({'no_data':[]})
    else:
        df = df_in.copy()
        if columns_to_display is not None:
            if any([c not in df.columns.values for c in columns_to_display]):
                m = f'{columns_to_display} are missing from input data. Your input Csv'
                raise BadColumnsException(m)           
            df = df[columns_to_display]
            
    dt.data=df.to_dict('rows')
    dt.columns=[{"name": i, "id": i,'editable': True if i in editable_columns else False} for i in df.columns.values]                    
    s = h4_like if title_style is None else title_style
    child_div = html.Div([html.Div(html.Div(title,style=s),style=table_like),dt])
    lg.debug(f'{dtable_id} exiting create_dt_div')
    return child_div


In [10]:
def create_xy_graph():
    pass

In [11]:
def recursive_grid_layout(app_component_list,current_component_index,gtcl,layout_components,wrap_in_loading_state=False):
    # loop through the gtcl, and assign components to grids
    for grid_template_columns in gtcl:
        if type(grid_template_columns)==list:
            layout_components.append(recursive_grid_layout(
                app_component_list,current_component_index, grid_template_columns, 
#                 layout_components,wrap_in_loading_state=True))
                layout_components,wrap_in_loading_state=False))
            continue
        # if this grid_template_columns item is NOT a list, process it normally
        sub_list_grid_components = []
        num_of_components_in_sublist = len(grid_template_columns.split(' '))
        for _ in range(num_of_components_in_sublist):
            # get the current component
            layout_ac = app_component_list[current_component_index]
            # add either the component, or it's html property to the sublist
            if hasattr(layout_ac, 'html'):
                layout_ac = layout_ac.html
            sub_list_grid_components.append(layout_ac)
            current_component_index +=1
        new_grid = create_grid(sub_list_grid_components, 
                        additional_grid_properties_dict={'grid-template-columns':grid_template_columns},
                        wrap_in_loading_state=wrap_in_loading_state)
        layout_components.append(new_grid)

def make_layout(comp_list,grid_template_columns_list):
    # for horizontal default
#     default_gtcl = [' '.join(['1fr' for _ in range(len(comp_list))])]
    # for vertical default
    default_gtcl = ['1fr' for _ in range(len(comp_list))]
    gtcl = default_gtcl if grid_template_columns_list is None else grid_template_columns_list
    
    # populate layout_components using recursive algo
    layout_list = []    
    recursive_grid_layout(comp_list,0,gtcl,layout_list)
    return    layout_list

In [12]:
class_converters = {
    dcc.Checklist:lambda v:v,
    dcc.DatePickerRange:lambda v:v,
    dcc.DatePickerSingle:lambda v:v,
    dcc.Dropdown:lambda v:v,
    dcc.Input:lambda v:v,
    dcc.Markdown:lambda v:v,
    dcc.RadioItems:lambda v:v,
    dcc.RangeSlider:lambda v:v,
    dcc.Slider:lambda v:v,
    dcc.Store:lambda v:v,
    dcc.Textarea:lambda v:v,
    dcc.Upload:lambda v:v,
}

html_members = [t[1] for t in inspect.getmembers(html)]
dcc_members = [t[1] for t in inspect.getmembers(dcc)]
all_members = html_members + dcc_members

class DashLink():
    def __init__(self,in_tuple_list, out_tuple_list,io_callback=None,state_tuple_list= None):
        _in_tl = [(k.id if type(k) in all_members else k,v) for k,v in in_tuple_list]
        _out_tl = [(k.id if type(k) in all_members else k,v) for k,v in out_tuple_list]

        self.inputs = [Input(k,v) for k,v in _in_tl]
        self.outputs = [Output(k,v) for k,v in _out_tl]
        
        self.states = [] 
        if state_tuple_list is not None:
            _state_tl = [(k.id if type(k) in all_members else k,v) for k,v in state_tuple_list]
            self.states = [State(k,v) for k,v in _state_tl]
        
        self.io_callback = lambda input_list:input_list[0] 
        if io_callback is not None:
            self.io_callback = io_callback
                       
    def callback(self,theapp):
        @theapp.callback(
            self.outputs,
            self.inputs,
            self.states
            )
        def execute_callback(*inputs_and_states):
            l = list(inputs_and_states)
            ret = self.io_callback(l)
            return ret if type(ret) is list else [ret]
        return execute_callback
        
def makeapp(comp_list,dlink_list,grid_layout_list=None):
    app = dash.Dash()
    layout_components = make_layout(comp_list,grid_layout_list)
    grid_html = html.Div(layout_components,style={'margin-left':'10px','margin-right':'10px'})
    app.layout=html.Div(grid_html)
    for dlink in dlink_list:
        dlink.callback(app)
    return app

In [13]:
def generate_dataframe(column_names,possible_values_per_name,weights_per_name,num_rows):
    """
    :param column_schema: a list of dict objects where each element is:
        key/value: 'column_name':name
        key/value: 'possible_values':list of possible values
        key/value: 'weights_per_value': list of weights
        
    """
    col_data_lists = [] # this is a 2-d list
    for i in range(len(column_names)):
        possible_values = possible_values_per_name[i]
        weights = weights_per_name[i]
        choices = random.choices(possible_values,weights=weights,k=num_rows)
        col_data_lists.append(choices)
    # assemble dataframe
    data = {column_names[i]:col_data_lists[i] for i in range(len(column_names))}
    return pd.DataFrame(data)

In [14]:
def generate_random_walk_df(
        num_days_to_create=1000,
        end_date = None,
        annual_std_for_close=.15, 
        annual_std_for_volume=.5,
        initial_volume=1000000):
    all_days=num_days_to_create
    end_date = datetime.datetime.now() if end_date is None else end_date
    beg_date = end_date - datetime.timedelta(all_days)
    date_series = pd.bdate_range(beg_date,end_date)
    trade_dates = date_series.astype(str).str.replace('-','').astype(int)
    n = len(trade_dates)
    changes = np.random.lognormal(0,annual_std_for_close/256**.5,n-1)
    initial = np.array([100.0])
    closes = np.cumprod(np.append(initial,changes)).round(2)
    open_ranges = np.random.lognormal(0,.3/256**.5,n)
    opens = (closes * open_ranges).round(2)
    low_ranges = np.random.lognormal(.1,.2/256**.5,n)
    lows = np.array([min(x,y) for x,y in zip(opens,closes)]) - low_ranges
    lows = lows.round(2)
    high_ranges = np.random.lognormal(.1,.2/256**.5,n)
    highs = np.array([max(x,y) for x,y in zip(opens,closes)]) + high_ranges
    highs = highs.round(2)

    volume_changes = np.random.lognormal(0,annual_std_for_volume/256**.5,n-1)
    initial_volumes = np.array([initial_volume]) * np.random.lognormal(0,annual_std_for_volume/256**.5)
    volumes = np.cumprod(np.append(initial_volumes,volume_changes)).round(0)


    df_pseudo = pd.DataFrame({'date':trade_dates,'open':opens,'high':highs,'low':lows,
                      'close':closes,'volume':volumes})
    return df_pseudo



In [15]:
def generate_random_breakfast_times():
    
    rows = 40
    col_names = ['name','day','breakfast_time','minute']

    names = ["Sarah", "Billy", "Michael"]
    days = ["Monday", "Tuesday","Wednesday",'Thursday','Friday']
    times = list(range(7,12))
    minutes = list(range(9,17))
    possible_values_list = [names,days,times,minutes]

    name_weights = [2, 1, 1]
    day_weights = [2,1,2,1,1]
    time_weights = np.ones(len(times))
    minute_weights = np.ones(len(minutes))
    weights_list = [name_weights,day_weights,time_weights,minute_weights]

    df2 = generate_dataframe(col_names,possible_values_list,weights_list,rows)
    return df2

### Example Apps

In [16]:
def pick_from_random_lists_app():
    df2 = generate_random_breakfast_times()

    rr1_ops = choices_from_df(df2,[None]*3,df2.columns.values[0])
    rr1 = dcc.RadioItems('rr1',options=rr1_ops[1],value=rr1_ops[0])

    dd1 = dcc.Dropdown('dd1',options=[])
    dd1_link = DashLink([(rr1.id,'value')],[(dd1.id,'value'),(dd1.id,'options')],
                      lambda input_list:list(choices_from_df(df2,input_list,df2.columns.values[1])))

    rr2 = dcc.RadioItems('rr2')
    rr2_link = DashLink([(rr1.id,'value'),(dd1.id,'value')],[(rr2.id,'value'),(rr2.id,'options')],
                      lambda input_list:list(choices_from_df(df2,input_list,df2.columns.values[2])))

    dd2 = dcc.Dropdown('dd2',options=[])
    dd2_link = DashLink([(rr1.id,'value'),(dd1.id,'value'),(rr2.id,'value')],[(dd2.id,'value'),(dd2.id,'options')],
                      lambda input_list:list(choices_from_df(df2,input_list,df2.columns.values[3])))

    dt1 = create_dt_div('dt1',df_in=df2,title='My Table')
    dt1_link = DashLink([(rr1.id,'value'),(dd1.id,'value'),(rr2.id,'value')],[('dt1','data')],
                      lambda input_list:[slice_df(df2,input_list).to_dict('records')])
    
    gtcl = ['1fr 1fr 1fr 1fr','1fr']
    app = makeapp([rr1,dd1,rr2,dd2,dt1],[dd1_link,rr2_link,dd2_link,dt1_link],gtcl)
    app.run_server(host='127.0.0.1',port=8500)
   

In [17]:
def value_componets_example():
    days = ["Monday", "Tuesday","Wednesday",'Thursday','Friday']
    ch1 = dcc.Checklist(id='ch1',options=[{'label':v,'value':v} for v in days],value=[days][2:4])
    sl1 = dcc.RangeSlider(id='sl1',min=-5,max=10,step=0.5,value=[-3,5])
    
    div1 = html.Div(children=[],id='div1')    
    div1_link = DashLink([(ch1,'value'),(sl1,'value')],[(div1,'children')],
                         lambda input_list:' , '.join([str(v) for v in input_list]))

    gtcl = ['1fr 1fr 1fr']
    app = makeapp([ch1,sl1,div1],[div1_link],gtcl)
    app.run_server(host='127.0.0.1',port=8500)


In [18]:
def random_market_data_app():
    dfr = generate_random_walk_df()
    lb1 = html.Label("Enter Begin yyyymmdd (e.g. 20180524)")
    in1 = dcc.Input(id='in1',placeholder='Enter a begining yyyymmdd',type='number',value=dfr.date.values[-50],debounce=True)
    lb2 = html.Label("Enter Ending yyyymmdd (e.g 20181124)")
    in2 = dcc.Input(id='in2',placeholder='Enter a ending yyyymmdd',type='number',value=dfr.date.values[-1],debounce=True)
    dt1 = create_dt_div('dt1',df_in=dfr,title='My Table')
    
    dt1_link = DashLink(
        [(in1,'value'),(in2,'value')],
        [('dt1','data')],
#         lambda input_list:[dfr[(dfr.date>=input_list[0][0]) & (dfr.date<=input_list[1][0])].to_dict('records')])
        lambda input_list:[dfr[(dfr.date>=input_list[0]) & (dfr.date<=input_list[1])].to_dict('records')])

    def _makegraph(input_list):
        dict_df = input_list[0]
        df = pd.DataFrame(dict_df)
        fig1 = plc(df,number_of_ticks_display=20,title='Random Walk').get_figure()
        gr1 = dcc.Graph(id='gr1',figure=fig1)
        return gr1
        
    div1 = html.Div(children=[],id='div1')
    div1_link = DashLink([('dt1','data')],[('div1','children')],_makegraph)
                                         
    gtcl = ['1fr 1fr 1fr 1fr','1fr', '1fr']
    app = makeapp([lb1,in1,lb2,in2,div1,dt1],[dt1_link,div1_link],gtcl)
    app.run_server(host='127.0.0.1',port=8500)


In [19]:
def dropdown_to_df_app():
    dfr = generate_random_breakfast_times()
    dd_names = dcc.Dropdown(id='dd_names',options=choices_from_df(dfr,[None]*3,dfr.columns.values[0])[1])
    dd_days = dcc.Dropdown(id='dd_days',options=choices_from_df(dfr,[None]*3,dfr.columns.values[1])[1])
    dd_times = dcc.Dropdown(id='dd_times',options=choices_from_df(dfr,[None]*3,dfr.columns.values[2])[1])
    
    dt1 = create_dt_div('dt1',df_in=dfr,title="Breakfast times")
    def _slice_dfr(input_list):
        dfr2 = dfr.copy()
        cols =  dfr2.columns.values
        for i in range(len(input_list)):
            dfr2 = dfr2 if input_list[i] is None else dfr2[dfr2[cols[i]]==input_list[i]]
        return [dfr2.to_dict('records')]

    dt1_link = DashLink([(dd_names,'value'),(dd_days,'value'),(dd_times,'value')],[('dt1','data')],_slice_dfr)


    def _get_choices(input_list,col_num):
        dict_df = input_list[0]
        if dict_df is None:
            return None
        dft = pd.DataFrame(dict_df)
        col = dft.columns.values[col_num]
        value_and_choices = choices_from_df(dft,[None,None,None],col)
        ret =  [value_and_choices[1]]
        return ret

    dd_names_link = DashLink([('dt1','data')],[(dd_names,'options')],lambda input_list:_get_choices(input_list,0))
    dd_days_link = DashLink([('dt1','data')],[(dd_days,'options')],lambda input_list:_get_choices(input_list,1))
    dd_times_link = DashLink([('dt1','data')],[(dd_times,'options')],lambda input_list:_get_choices(input_list,2))

    gtcl = ['1fr 1fr 1fr','1fr']
    app = makeapp([dd_names,dd_days,dd_times,dt1],[dt1_link,dd_names_link,dd_days_link,dd_times_link],gtcl)
    app.run_server(host='127.0.0.1',port=8500)
    

In [20]:
def make_cte_lines(col_regex_list,cols,schema,table):
    from_name = f'{schema}.{table}'
    select_statements = []
    cte_names = []
    for i in range(len(col_regex_list)):
        r = col_regex_list[i]
        if r is None or len(str(r).strip())<1:
            continue
        c = cols[i]
        verb = '~*'
        if all([ch not in r for ch in ['*','^']]):
            verb = '='
        s = f"select * from {from_name} where {c} {r}"
        select_statements.append(s)
        cte_name = f'f_{c}'
        cte_names.append(cte_name)
        from_name = cte_name
     
    cte_lines = [] if len(select_statements) < 2 else ['with ']
    for i in range(len(select_statements)):
        s = select_statements[i]
        if i< len(select_statements)-1:
            s = f"{cte_names[i]} as ({s})"
            if i > 0:
                s = ',' + s
        cte_lines.append(s)
        
    return cte_lines

def make_cte_div(col_regex_list,cols,schema,table):
    cte_lines = make_cte_lines(col_regex_list,cols,schema,table)
    full_cte = html.Div([html.Div(cte_line) for cte_line in cte_lines])
    return full_cte

def get_cte_df(col_regex_list,cols,pga,schema,table):
    full_cte_sql = f'select * from {schema}.{table} limit 50'
    cte_lines = make_cte_lines(col_regex_list,cols,schema,table)
    if cte_lines is not None and len(cte_lines)>0:
        full_cte_sql = '\n'.join(cte_lines)
    full_cte_sql = full_cte_sql.replace('%','%%')
    df = pga.get_sql(full_cte_sql)
    return [df.to_dict('records')]

In [21]:
def sql_cte_buileder_example_sec():
    sql_cte_builder_example('local','sec_schema','options_table')


def sql_cte_builder_example(db_info_name,schema,table):
    pga = dbi.get_db_info(db_info_name)
    cols = pga.get_sql(f"select * from {schema}.{table} limit 1").columns.values

    # set up top title 
    title_markedown = """
    # Postgres Sql Common Table Expression Builder
    * Use the left panel to enter sql-like where criteria for any column
    * Watch the CTE being built
    * Click on the execute cte button to run the CTE query and return data
    """
    mk1 = dcc.Markdown(title_markedown,id='mk1',style={'color':'black','textAlign': 'center',})
    # set up labels and input boxes for all columns
    labels = [html.Label(c,id=f'{c}_label') for c in cols]
    inputs = [dcc.Input(id=f'{c}_input',debounce=True) for c in cols]
    pair_style = {'display': 'grid','grid-template-columns': '30% 70%'}
    label_input_pairs = [html.Div(children=[html.Div(children=[labels[i],inputs[i]],style=pair_style)],id=f'{cols[i]}_div') for i in range(len(cols))] 
    # get initial rows to display
    df_initial_data = pga.get_sql(f"select * from {schema}.{table} limit 50")

    # create the DashTable div that shows the data
    dt1 = create_dt_div('dt1',df_in=df_initial_data,title=html.H3('Commodity Options Table'))
    # wrap that div in a Loading component so that the user can see when he is waiting
    dt1_cube = dcc.Loading(html.Div(dt1),type='cube')
    
    # create the div that shows the actual cte
    cte_div = html.Div(id='cte_div')
    cte_in_tuples = [(inp,'value') for inp in inputs]
    cte_out_tuples = [(cte_div,'children')]
    # create the DashLink that connects the input with the output cte div
    cte_link = DashLink(
        cte_in_tuples,
        cte_out_tuples,
        lambda input_list:make_cte_div(input_list,cols,schema,table))

    # create the run sql button
    run_sql_button = html.Button('Click to Run sql CTE',id='run_sql')
    # connect the button to the DashTable, and to the call to the get_cte_df callback
    dt1_link = DashLink(
        [(run_sql_button,'n_clicks')],
        [('dt1','data')],
        lambda input_list:get_cte_df(input_list if len(input_list)<2 else input_list[1:],cols,pga,schema,table),
        state_tuple_list=cte_in_tuples)

    # create the divs that follow the label_input_pairs divs
    other_inputs = [html.P(),run_sql_button,html.P(),
                    html.H3('CTE for sql',style=cte_title_style),cte_div]
    # merge all divs that go on the left column together
    input_div_title = html.H3('Enter where-like criteria below:',style=cte_title_style)
    input_div_children = [input_div_title] + label_input_pairs + other_inputs
    input_div = html.Div(children=input_div_children,id='input_div')
    
    # create the layout grid
    gtcl = ['1fr','30% 70%']
    # make and run the app
    app = makeapp([mk1,input_div,dt1_cube],[cte_link,dt1_link],gtcl)
    app.run_server(host='127.0.0.1',port=8500)


    

In [22]:
class SqlExpression():
    """
    Build and sql expresssion that can also be fed into a CTE.
    Example:
    
    """
    def __init__(self,main_table_name,main_table_alias=None):
        self.main_table_name = main_table_name
        self.main_table_alias = main_table_name if main_table_alias is None else main_table_alias
        self.join_expressions = []
        self.where_clause_list = []
        self.regular_col_list = []
        self.aggregate_col_list = []
        self.sort_order_list = []    
    
    def get_main_table(self):
        return self.main_table_name
    
    def get_main_table_alias(self):
        return self.main_table_alias

    def add_join_expression(self,prev_table_name,prev_table_alias,on_expression_list):
        on_expression_str = ' and '.join(on_expression_list)
        join_expression = f"{prev_table_name} {prev_table_alias} on {on_expression_str}"
        self.join_expressions.append(join_expression)

    def add_where_clause(self,where_col, where_verb,where_predicate):
        wp = where_predicate.replace('%','%%') if (type(where_predicate) == str) else where_predicate
        wv = f" {where_verb} "
        where_expression = f"{where_col} {wv} {wp}"
        self.where_clause_list.append(where_expression)

    def add_regular_display_cols(self,col_list):
        self.regular_col_list.extend(col_list)
        
    def add_aggregate_display_cols(self,agg_expression_list):
        self.aggregate_col_list.extend(agg_expression_list)

    def add_sort_cols(self,col_list):
        self.sort_order_list.extend(col_list)
    
    def make_expression(self):
        join_str = '' if len(self.join_expressions)<1 else 'join ' + ' join '.join(self.join_expressions)
        where_str = '' if len(self.where_clause_list)<1 else 'where ' + ' and '.join(self.where_clause_list)
        reg_col_str = '*' if len(self.regular_col_list)<1 else ','.join(self.regular_col_list)
        agg_col_str = ','.join(self.aggregate_col_list)
        all_col_str = reg_col_str + ('' if len(agg_col_str)<1 else ',' + agg_col_str)
        group_by_str = '' if len(agg_col_str)<1 else f'group by {reg_col_str}'
        order_by_str = '' if len(self.sort_order_list) < 1 else 'order by ' + ','.join(self.sort_order_list)
        expression = f"""
        select {all_col_str} 
        from {self.main_table_name} {self.main_table_alias}
        {join_str}
        {where_str}
        {group_by_str}
        {order_by_str}
        """
        return expression
    
class CteBuilder():
    def __init__(self,sql_expression_list,limit=20):
        self.sql_expression_list = sql_expression_list
        self.limit = limit
        
    def make_sql(self):
        sl = self.sql_expression_list
        sle = [s.make_expression() for s in sl]
        full_sql = 'with \n'
        full_sql += '\n,'.join([f'f{i} as (' + sle[i] + ')' for i in range(len(sle))])
        last_cte_name = str(len(sle)-1)
        lm = '' if self.limit is None else f' limit {self.limit}'
        full_sql += f"\nselect * from f{last_cte_name} {lm}"
        return full_sql

In [23]:
def two_inputs_example():
    inp_a = dcc.Input(value=1,id='inp_a')
    inp_b = dcc.Input(value=11,id='inp_b')
    inps_link = DashLink([(inp_a,'value')],[(inp_b,'value')],lambda input_list:input_list[0])
    gtcl = ['1fr 1fr']
    app = makeapp([inp_a,inp_b],[inps_link],gtcl)
    app.run_server(host='127.0.0.1',port=8500)


### Run examples

In [ ]:

if __name__=='__main__':
    app_list = [
        pick_from_random_lists_app,
        random_market_data_app,
        value_componets_example,
        dropdown_to_df_app,
        sql_cte_buileder_example_sec,
        two_inputs_example
    ]
    app_list[4]()


In [ ]:
# !jupyter nbconvert --to script easycomp.ipynb

## Define CteBuilder GUI

In [ ]:
# ot = 'df_ot'#'sec_schema.options_table' # ot = options table
# ut = 'sec_schema.underlying_table'
df_ot = pd.read_csv('./df_options_table_sample.csv')
df_ut = pd.read_csv('./df_underlying_table_sample.csv')
ot = 'df_ot'#'sec_schema.options_table' # ot = options table
ut = 'df_ut'#'sec_schema.underlying_table'

sqle1 = SqlExpression(ot,'ot')
# add join, where, reg, agg, sort
sqle1.add_join_expression(ut,'ut',['ot.symbol=ut.symbol','ot.settle_date=ut.settle_date'])
# sqle1.add_where_clause('ut.symbol','like',"'CLF19'")
sqle1.add_where_clause('ut.symbol','like',"'CLF%'")
sqle1.add_where_clause('ut.volume','>',0)
sqle1.add_where_clause('ut.open_interest','>',0)
sqle1.add_regular_display_cols(['ot.*','ut.close future'])

sqle2 = SqlExpression('f0')
sqle2.add_regular_display_cols(['f0.symbol','f0.settle_date'])
sqle2.add_aggregate_display_cols([
    'sum(f0.volume) vol_sum', 'sum(f0.open_interest) oi_sum', 
    'max(f0.volume) max_vol', 'max(f0.open_interest) max_oi'])

sqle3 = SqlExpression('f0')
sqle3.add_join_expression('f1','f1',['f0.symbol=f1.symbol','f0.settle_date=f1.settle_date'])
sqle3.add_regular_display_cols([
    'f0.*','cast(f0.volume as float)/f1.vol_sum vol_perc',
    'cast(f0.open_interest as float)/f1.oi_sum oi_perc'])
sqle3.add_where_clause('f0.volume','>',0)
sqle3.add_where_clause('f0.open_interest','>',0)

pga = dbi.get_db_info('local')
sql = CteBuilder([sqle1,sqle2,sqle3],limit=None).make_sql()
df_sql_results = None
try:
#     df = pga.get_sql(sql)
    df_sql_results = sqldf(sql,globals())
except Exception as e:
    print(e)
print(sql)



In [ ]:
df_sql_results


In [ ]:
# df_ot = pga.get_sql("select * from sec_schema.options_table where symbol like 'CLF%%'")
# df_ut = pga.get_sql("select * from sec_schema.underlying_table where symbol like 'CLF%%'")
# df_ot.to_csv('./df_options_table_sample.csv',index=False)
# df_ut.to_csv('./df_underlying_table_sample.csv',index=False)

In [ ]:
# header markdown div
# file input: 
# editable table showing table names, and aliases

# area for building sql
# main table alias
# add join statements
# add where statements
# add groupby statements
# add orderby statements

# area for putting sql statements into cte

# run cte

# table display of dataframe

# build graph area

### title

In [76]:
# set up top title 
title_markedown = """
# Postgres Sql Common Table Expression Builder
* Use the left panel to enter sql-like where criteria for any column
* Watch the CTE being built
* Click on the execute cte button to run the CTE query and return data
"""
mk1 = dcc.Markdown(title_markedown,id='mk1',style={'color':'black','textAlign': 'center',})


### upload tables and display dataframe of tables

In [79]:
upload1 = dcc.Upload(
            id='upload1',
            children=html.Div("Choose csv file:"),
            accept = '.csv',
            # Allow multiple files to be uploaded
            multiple=True,
            style=blue_button_style)

dfcsv_init = pd.DataFrame({'table_name':[],'alias':[],'columns':[]})
def make_dtcsv_table(input_list):
#     print(f'make_dt_csv_table {input_list}')
    if input_list is None or len(input_list)<1 or input_list[0] is None:
        return [dfcsv_init.to_dict('records')]
    file_names = input_list[0]
    file_names = [s.replace('.csv','') for s in file_names]
    aliases = [f'table_{n}' for n in range(len(file_names))]
    
    # get contents columns so that you can display available columns to query with sql
    contents_list = input_list[1]    
    list_df = [parse_contents(content) for content in contents_list]
    col_lists = [','.join(df.columns.values) for df in list_df]
    dft = pd.DataFrame({'table_name':file_names,'alias':aliases,'columns':col_lists})
    r = [dft.to_dict('records')]
    return r

# create the DashTable div that shows the data
dtcsv = create_dt_div('dtcsv',df_in=dfcsv_init,title=html.H4('Input Tables'),editable_columns_in=['alias'])
# create store to hold dataframes
s1 = dcc.Store(id='s1')
# make link to update list of tables that are used in sql calls
dtcsv_link = DashLink([(upload1,'filename')],[('dtcsv','data')],make_dtcsv_table,state_tuple_list=[(upload1,'contents')])



2020-02-27 13:14:00,599 - root - DEBUG - dtcsv entering create_dt_div
2020-02-27 13:14:00,609 - root - DEBUG - dtcsv exiting create_dt_div


### make link that stores all csv data for sql calls and build a Loading component to display the tables that were loaded by csv

In [86]:

def make_s1(input_list):
    if input_list is None or len(input_list)<1 or input_list[0] is None:
        stop_callback('make_s1 no data',logger)
    # make dataframe dicts
    contents_list = input_list[0]    
    list_df = [parse_contents(content) for content in contents_list]
    
    file_names = input_list[1]
    file_names = [s.replace('.csv','') for s in file_names]
    dict_df = {file_names[i]:list_df[i].to_dict('records') for i in range(len(list_df)) }
    return [dict_df]
s1_link = DashLink([(upload1,'contents')],[(s1,'data')],make_s1,state_tuple_list=[(upload1,'filename')])
# build a div with the table and the store
dtcsv_cube = dcc.Loading(html.Div([dtcsv,s1]),type='cube')


### main table selection dropdowns

In [78]:
# select main table
main_dropdown = dcc.Dropdown(id='main_dropdown')
main_dropdown_div = html.Div(['Select a main table from the dropdown:'],id='main_dropdown_div')
def populate_main_dropdown_options(input_list):
    if input_list is None or len(input_list)<1 or input_list[0] is None:
        stop_callback('populate_main_dropdown_options no data',logger)
    dict_df = input_list[0]
    if len(dict_df)<1:
        stop_callback('populate_main_dropdown_options no data',logger)
    df_tables = make_df(dict_df)
    table_names = df_tables.table_name.values
    ret = [{'label':tn,'value':tn} for tn in table_names]
    return [ret]
main_dropdown_link = DashLink(
    [('dtcsv','data')],[(main_dropdown,'options')],populate_main_dropdown_options)


### create left and right join table selections

In [80]:
# define dcc component, div and DashLink for selecting left table
join_left_radioitems = dcc.RadioItems(id='join_left_radioitems',labelStyle={"display":"block","vertical-align":"middle"})
join_left_radioitems_div = html.Div([html.Div('Select left join table:'),html.Div([join_left_radioitems])],id='join_left_radioitems_div')
def populate_join_left_radio_options(input_list):
    if input_list is None or len(input_list)<1 or input_list[0] is None:
        stop_callback('populate_join_radio_options no data',logger)
    main_table_name = input_list[0]
    all_table_names = [d['value'] for d in input_list[1]]
#     join_table_names = [jt for jt in all_table_names if jt != main_table_name]
    join_table_names = all_table_names
    ret = [{'label':tn,'value':tn} for tn in join_table_names] #+ [{}]
    return [ret]
join_left_radioitems_link = DashLink(
    [(main_dropdown,'value')],[(join_left_radioitems,'options')],
    populate_join_left_radio_options,state_tuple_list=[(main_dropdown,'options')])

# define dcc component, div and DashLink for selecting right table
join_right_radioitems = dcc.RadioItems(id='join_right_radioitems',labelStyle={"display":"block","vertical-align":"middle"})
join_right_radioitems_div = html.Div([html.Div('Select right join table:'),html.Div([join_right_radioitems])],id='join_right_radioitems_div')
def populate_join_right_radio_options(input_list):
    if input_list is None or len(input_list)<1 or input_list[0] is None:
        stop_callback('populate_join_radio_options no data',logger)
    main_table_name = input_list[0]
    all_table_names = [d['value'] for d in input_list[1]]
    join_table_names = [jt for jt in all_table_names if jt != main_table_name]
    ret = [{'label':tn,'value':tn} for tn in join_table_names] #+ [{}]
    return [ret]
join_right_radioitems_link = DashLink(
    [(join_left_radioitems,'value')],[(join_right_radioitems,'options')],
    populate_join_right_radio_options,state_tuple_list=[(join_left_radioitems,'options')])


### Define 3 sets of columns to use for left and right join

In [83]:
# define dcc component, div and DashLink for selecting left table columns
def populate_join_columns_dropdown_options(input_list):
    if input_list is None or len(input_list)<1 or input_list[0] is None:
        stop_callback('populate_join_columns_dropdown_options no left table_name data',logger)
    table_name = input_list[0]
    dict_df = input_list[1]
    if len(dict_df)<1:
        stop_callback('populate_join_columns_dropdown_options no dataframe data',logger)
    df_tables = make_df(dict_df)
    df_tables_this_table = df_tables[df_tables.table_name==table_name]
    if (len(df_tables_this_table)<1):
        stop_callback(f'populate_join_columns_dropdown_options no data for table {table_name}',logger)
    columns = str(df_tables_this_table.iloc[0]['columns']).split(',')
    ret = [{'label':c,'value':c} for c in columns]
    return [ret]

class ColumnsDropdown():
    def __init__(self,id_base,join_left_radioitems,join_right_radioitems):
        # define left
        self.verb_options = [{'label':v,'value':v} for v in ['=','<','>','<=','>=','like','ilike']]
        self.join_left_columns_dropdown = dcc.Dropdown(id=f'join_left_columns_dropdown_{id_base}')
        self.join_left_columns_dropdown_div = html.Div([html.Div('Select left join column:'),html.Div([self.join_left_columns_dropdown])],
                                                       id=f'join_left_columns_dropdown_div_{id_base}')
        self.join_left_columns_dropdown_link = DashLink(
            [(join_left_radioitems,'value')],[(self.join_left_columns_dropdown,'options')],
            populate_join_columns_dropdown_options,state_tuple_list=[('dtcsv','data')])
        
        # define right
        self.join_right_columns_dropdown = dcc.Dropdown(id=f'join_right_columns_dropdown_{id_base}')
        self.join_right_columns_dropdown_div = html.Div([html.Div('Select right join column:'),html.Div([self.join_right_columns_dropdown])],
                                                       id=f'join_right_columns_dropdown_div_{id_base}')
        self.join_right_columns_dropdown_link = DashLink(
            [(join_right_radioitems,'value')],[(self.join_right_columns_dropdown,'options')],
            populate_join_columns_dropdown_options,state_tuple_list=[('dtcsv','data')])
        
        # define dcc component, div and DashLink for selecting join verb
        self.join_verb_dropdown = dcc.Dropdown(id=f'join_verb_dropdown_{id_base}',value='=',options=self.verb_options,placeholder="select verb")
        self.join_verb_dropdown_div = html.Div([html.Div(['Select a join verb:']),html.Div(self.join_verb_dropdown)],id=f'join_verb_dropdown_div_{id_base}')
        self.div = html.Div(children=[
            self.join_left_columns_dropdown_div,
            self.join_verb_dropdown_div,
            self.join_right_columns_dropdown_div
        ])
        self.links = [self.join_left_columns_dropdown_link,
                     self.join_right_columns_dropdown_link]

# create a button that create the join sql from the dropdowns
cd1 = ColumnsDropdown('cd1',join_left_radioitems,join_right_radioitems)
# cd1_div = cd1.div
# cd1_links = cd1.links
cd2 = ColumnsDropdown('cd2',join_left_radioitems,join_right_radioitems)
# cd2_div = cd2.div
# cd2_links = cd2.links
cd3 = ColumnsDropdown('cd3',join_left_radioitems,join_right_radioitems)
# cd3_div = cd3.div
# cd3_links = cd3.links


### create the div tha holds the join statements, and get's updated via the join_add_button

In [88]:
join_add_button = html.Button('Add join', id='join_add_button')
sqlin = dcc.Textarea(id='sqlin')
join_store = dcc.Store(id='join_store',data=[])
def update_join_store(input_list):
    if input_list is None or len(input_list)<1 or input_list[0] is None:
        stop_callback('update_join_store no data',logger)
    join_list = input_list[1]
    left_table = input_list[2]
    if any([v is None for v in [left_table,join_list]]):
        stop_callback('update_join_store join info missing',logger)
    right_table = input_list[3]
    join_text = ''
    if input_list[4] is  None:
        stop_callback('update_join_store no column info specified',logger)
    join_text = f"join {left_table} on "
    for i in range(3): 
        if input_list[4+i*3] is not None:
            next_join_clause = f"{left_table}.{input_list[4+i*3]} {input_list[5+i*3]} {left_table}.{input_list[6+i*3]}"
            if i>0:
                next_join_clause = ' and ' + next_join_clause
            join_text += next_join_clause
                    
    # assemble a new join from radioitems, checklists and dropdowns
    join_list.append(join_text)
    return [join_list]

join_store_link = DashLink([(join_add_button,'n_clicks')],[(join_store,'data')],
                          update_join_store,
                          state_tuple_list=[
                              (join_store,'data'),
                              (join_left_radioitems,'value'),
                              (join_right_radioitems,'value'),
                              (cd1.join_left_columns_dropdown,'value'),
                              (cd1.join_verb_dropdown,'value'),
                              (cd1.join_right_columns_dropdown,'value'),
                              (cd2.join_left_columns_dropdown,'value'),
                              (cd2.join_verb_dropdown,'value'),
                              (cd2.join_right_columns_dropdown,'value'),
                              (cd3.join_left_columns_dropdown,'value'),
                              (cd3.join_verb_dropdown,'value'),
                              (cd3.join_right_columns_dropdown,'value')
                          ])
def update_sqlin(input_list):
    if input_list is None or len(input_list)<1 or input_list[0] is None:
        stop_callback('update_sqldiv no data',logger)
    join_statements = input_list[0] 
    join_text = '\n'.join(join_statements)
    return [join_text]
sqldiv_update_link = DashLink([(join_store,'data')],[(sqlin,'value')],update_sqlin)


In [75]:
# sql input line 
# sqlin = dcc.Input(id='sqlin',debounce=True)

# define running sql
sql_run_button = html.Button('Execute SQL:',id='sql_run_button')
sqldiv = html.Div(children=[],id='sqldiv')
sqlloading = dcc.Loading(sqldiv,type='cube')
def make_sqldiv(input_list):
    if input_list is None or len(input_list)<3 or input_list[1] is None:
        stop_callback('make_sqldiv no data',logger)
    sql = input_list[1]
    dict_df = input_list[2]
    if dict_df is None:
        stop_callback('make_sqldiv no csv data selected',logger)
    for k in dict_df.keys():
        globals()[k] = make_df(dict_df[k])
    df_sql_results = sqldf(sql,globals())
    dt_sql_results = create_dt_div('dtsql',df_in=df_sql_results,title=html.H4('Input Tables'))
    return [dt_sql_results]
sqlin_link = DashLink([(sql_run_button,'n_clicks')],[(sqldiv,'children')],make_sqldiv,state_tuple_list=[(sqlin,'value'),(s1,'data')])

# define display components 
components = [mk1,upload1,dtcsv_cube, #3
              main_dropdown_div,main_dropdown, #2
              join_left_radioitems_div,join_right_radioitems_div, #2
              cd1.join_left_columns_dropdown_div,cd1.join_verb_dropdown_div,cd1.join_right_columns_dropdown_div,#3
              cd2.join_left_columns_dropdown_div,cd2.join_verb_dropdown_div,cd2.join_right_columns_dropdown_div,#3
              cd3.join_left_columns_dropdown_div,cd3.join_verb_dropdown_div,cd3.join_right_columns_dropdown_div,#3
              join_add_button,#1
              sqlin,sql_run_button,sqlloading,join_store]#3
# define links
main_links = [dtcsv_link,s1_link,sqlin_link,main_dropdown_link]
jt_links = [join_left_radioitems_link,join_right_radioitems_link]
cd_links = cd1.links + cd2.links + cd3.links 
sql_links = [join_store_link,sqldiv_update_link]
links = main_links + jt_links + cd_links + sql_links
# define layout css
gtcl = ['1fr','1fr', # 
        '1fr','1fr 1fr',
        '1fr 1fr', 
        '1fr 1fr 1fr','1fr 1fr 1fr','1fr 1fr 1fr',
        '1fr', 
        '1fr','1fr','1fr','1fr']
# create and run app
app = makeapp(components,links,gtcl)
app.run_server(host='127.0.0.1',port=8500)


2020-02-27 05:30:35,097 - root - DEBUG - dtcsv entering create_dt_div
2020-02-27 05:30:35,099 - root - DEBUG - dtcsv exiting create_dt_div


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2020-02-27 05:30:35,154 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
2020-02-27 05:30:37,137 - werkzeug - INFO - 127.0.0.1 - - [27/Feb/2020 05:30:37] "GET / HTTP/1.1" 200 -
2020-02-27 05:30:37,406 - werkzeug - INFO - 127.0.0.1 - - [27/Feb/2020 05:30:37] "GET /_dash-layout HTTP/1.1" 200 -
2020-02-27 05:30:37,408 - werkzeug - INFO - 127.0.0.1 - - [27/Feb/2020 05:30:37] "GET /_dash-dependencies HTTP/1.1" 200 -
2020-02-27 05:30:37,523 - werkzeug - INFO - 127.0.0.1 - - [27/Feb/2020 05:30:37] "POST /_dash-update-component HTTP/1.1" 200 -
2020-02-27 05:30:37,544 - root - DEBUG - ****************************** update_join_store no data ***************************************
2020-02-27 05:30:37,546 - werkzeug - INFO - 127.0.0.1 - - [27/Feb/2020 05:30:37] "POST /_dash-update-component HTTP/1.1" 204 -
2020-02-27 05:30:37,555 - root - DEBUG - ****************************** populate_join_radio_options no data ***************************************
2020-02-27 05:

# End